In [25]:
!pip install docx2pdf


In [132]:
import json
from typing import Dict, Tuple, Optional
import logging

from typing import Dict
from docx import Document
from docx import Document
from typing import Dict
import re
import docx
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor
import os
from docx2pdf import convert
from typing import Dict
from openai import OpenAI
import openai
import os
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt
from dotenv import load_dotenv

load_dotenv()
print() #key should now be available
openai.api_key = os.getenv('OPENAI_API_KEY')
client =  OpenAI(
    api_key= os.environ.get("OPENAI_API_KEY"),
organization='org-Qf0ZgkD7gMaJ8A44ouvmZkvt',
)
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [138]:
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

def add_hyperlink(paragraph, url, text):
    """
    Add a hyperlink to a paragraph in a docx document.
    :param paragraph: The paragraph to which the hyperlink will be added.
    :param url: The URL for the hyperlink.
    :param text: The text to display for the hyperlink.
    """
    # Create the relationship for the hyperlink
    part = paragraph._parent.part
    r_id = part.relate_to(
        url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True
    )

    # Create the hyperlink tag
    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    # Create the run for the hyperlink text
    new_run = OxmlElement("w:r")
    rPr = OxmlElement("w:rPr")
    rStyle = OxmlElement("w:rStyle")
    rStyle.set(qn("w:val"), "Hyperlink")  # Apply the "Hyperlink" style
    rPr.append(rStyle)
    new_run.append(rPr)
    text_element = OxmlElement("w:t")
    text_element.text = text
    new_run.append(text_element)
    hyperlink.append(new_run)

    # Append the hyperlink element to the paragraph
    paragraph._element.append(hyperlink)
    return paragraph


In [16]:
docx_path1="./Faseeh Resume GA11.docx"

In [20]:
# Example usage:
doc = Document(docx_path1)
cv_data = extract_cv_data(doc)

# Print extracted data in a structured way
for section, content in cv_data.items():
    print(f"\n{'='*50}\n{section.upper()}:")
    if isinstance(content, dict):
        for key, value in content.items():
            print(f"\n{key}: {value}")
    elif isinstance(content, list):
        for item in content:
            if isinstance(item, dict):
                for k, v in item.items():
                    print(f"\n{k}: {v}")
            else:
                print(f"\n{item}")
job_description = "Your job description here"
prompts = create_resume_prompts(cv_data, job_description)


NAME:

CONTACT:

email: mm9314@g.rit.edu

phone: +1 585 202 5217

LinkedIn: https://www.linkedin.com/in/mohammad-faseeh-ahmed/

Github: https://github.com/faseehahmed26

Portfolio: https://faseehahmed26.github.io/portfolio/

Kaggle: https://www.kaggle.com/faseeh001

Tableau: https://public.tableau.com/app/profile/faseeh5112

EDUCATION:

Rochester Institute of Technology, Rochester, NY, M.S in Data Science	     Aug 2023 - May 2025

Coursework: Neural Networks, Software Engineering for Data Science, Applied Statistics		   GPA: 3.84/4.00

Jawaharlal Nehru Technological University Hyderabad, B.Tech in Computer Science	   July 2018 - July 2022

Coursework: Data Structures and Algorithms, Computer Vision, Data Warehousing, NLP 	            GPA: 3.2/4.00

SKILLS:

Programming Languages: Java, Python, C++, R, JavaScript, Go, Julia,  Object Oriented Programming(Python, Java)

Frameworks: PyTorch, Keras, Scikit, Tensorflow, Groovy, Spark, Flask, React, React-Native, NodeJS,

Databases: SQL, Mon

In [23]:
prompts['skills']

"\n        Given the job description and current skills, select the most relevant ones for each category.\n        Keep the same categories but optimize content within each to match job requirements.\n        Each category should fit on one line (10-16 words).\n        \n        Job Description: Your job description here\n        Current Skills: {'Programming Languages': 'Java, Python, C++, R, JavaScript, Go, Julia,  Object Oriented Programming(Python, Java)', 'Frameworks': 'PyTorch, Keras, Scikit, Tensorflow, Groovy, Spark, Flask, React, React-Native, NodeJS,', 'Databases': 'SQL, MongoDB, SQLite, MySQL, NoSQL, PostgreSQL, DynamoDB, SAS', 'Technologies': 'Docker, Git, AWS, Azure, GCP, Kafka, JSON, Numpy, Pandas, MLflow, Postman, Tableau,Power BI, MS Excel', 'ML Algorithms/Techniques': 'Regression, Classification, Clustering, Recommender Systems, Deep Learning, NLP, A/B'}\n        \n        Return format: same skills dictionary structure with optimized content.\n        "

In [117]:
 # Initialize OpenAI client
docx_path = "./Faseeh Curriculum Vitae.docx"

# Load your resume
doc = Document(docx_path)
cv_data = extract_cv_data(doc)  # Using the extract_cv_data function from earlier

job_description = '''LinkedIn logo
LinkedIn
Share
Show more options
AI Engineer 
Sunnyvale, CA · 3 weeks ago · Over 100 applicants
$111K/yr - $183K/yrMatches your job preferences, minimum pay preference is 140000.  HybridMatches your job preferences, workplace type is Hybrid.  Full-timeMatches your job preferences, job type is Full-time.
21 connections work here · 10 company alumni work here · 32 school alumni work here


Am I a good fit?

Tailor my resume

How can I best position myself?

Easy Apply

Saved
Saved AI Engineer  at LinkedIn
Stand out to the employer by marking this job as a top choice when you apply. Learn more
About the job
LinkedIn is the world’s largest professional network, built to help members of all backgrounds and experiences achieve more in their careers. Our vision is to create economic opportunity for every member of the global workforce. Every day our members use our products to make connections, discover opportunities, build skills and gain insights. We believe amazing things happen when we work together in an environment where everyone feels a true sense of belonging, and that what matters most in a candidate is having the skills needed to succeed. It inspires us to invest in our talent and support career growth. Join us to challenge yourself with work that matters.

This role will be based in Sunnyvale, San Francisco, Bellevue or New York City.

At LinkedIn, we trust each other to do our best work where it works best for us and our teams. This role offers a hybrid work option, meaning you can work from home and commute to a LinkedIn office, depending on what’s best for you and when it is important for your team to be together. This is a full-time engineering role based in Sunnyvale, San Francisco, Bellevue or New York City.
LinkedIn's Machine Learning Engineers are both data/research scientists and software engineers, who develop and implement machine learning models and algorithms. Unlike other companies that separate these roles, our engineers work on projects from ideation to implementation. 

Responsibilities:
• Building and deploying relevance models that power our various recommender systems and personalized intent engines
• Keeping scalability and performance in mind through your design and engineering choices.
• You will explore novel approaches for machine learning ranking and modeling downstream impact, as well as new ways of creating new product features by leveraging the cutting edge machine learning techniques in Generative AI, NLP.
• You will work with big data and analyze tracking data to understand member preferences and identify opportunities for taking the products you work on to the next level.
• You will work directly with partner data science, product, and infrastructure teams to take your ideas from conception to production.

Basic Qualifications:
• Bachelor's degree in Computer Science or related technical field or equivalent technical experience
• 1+ years experience with machine learning, data mining, and information retrieval or natural language processing 
• 1+ year of industry experience in software design, development, and algorithm related solutions.
• 1+ years experience in programming languages such as Java, Python, etc.

Preferred Qualifications: 
● 1+ years of relevant work experience
● MS or PhD in Computer Science or related technical discipline
● Experience developing large scale systems
● Knowledge of recommendation systems
● Ability to diagnose technical problems, debug code, and automate routine tasks
● Analytical approach coupled with solid communication skills and a sense of ownership
• Experience in Machine Learning and Deep Learning
• Experience in Big Data 
• Strong technical background & Strategic thinking

Suggested Skills:
• Data Mining
• Data Coding
• Machine Learning

'''

print(cv_data)

linkedin  linkedin https://www.linkedin.com/in/mohammad-faseeh-ahmed/
https://www.linkedin.com/in/mohammad-faseeh-ahmed/
github github https://github.com/faseehahmed26
https://github.com/faseehahmed26
portfolio portfolio https://faseehahmed26.github.io/portfolio/
https://faseehahmed26.github.io/portfolio/
kaggle kaggle https://www.kaggle.com/faseeh001
https://www.kaggle.com/faseeh001
tableau tableau https://public.tableau.com/app/profile/faseeh5112
https://public.tableau.com/app/profile/faseeh5112
{'Name': 'Mohammad Faseeh Ahmed', 'Contact': {'email': 'mm9314@g.rit.edu', 'phone': '+1 585 202 5217', 'links': {'LinkedIn': 'https://www.linkedin.com/in/mohammad-faseeh-ahmed/', 'Github': 'https://github.com/faseehahmed26', 'Portfolio': 'https://faseehahmed26.github.io/portfolio/', 'Kaggle': 'https://www.kaggle.com/faseeh001', 'Tableau': 'https://public.tableau.com/app/profile/faseeh5112'}}, 'Education': ['Rochester Institute of Technology, Rochester, NY,  M.S in Data Science\t      Expected

In [256]:
 # Template styling

# Process and save resume
# docx_path, pdf_path = process_complete_resume(cv_data, job_description, client)
# print(f"Resume saved as:\nDOCX: {docx_path}\nPDF: {pdf_path}")
optimized_content = process_complete_resume(cv_data, job_description, client)
# print(f"Resume saved as:\nDOCX: {docx_path}\nPDF: {pdf_path}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Optimized Content
{'skills': '```json\n{\n    "Programming Languages": "Java, Python",\n    "Frameworks": "PyTorch, Keras, Scikit-learn, TensorFlow, PySpark, Flask",\n    "Databases": "SQL, NoSQL, MongoDB, DynamoDB",\n    "Technologies": "Big Data",\n    "ML Algorithms/Techniques": "Recommender Systems, Deep Learning, NLP, Regression, Classification, Clustering"\n}\n```', 'experience': '```json\n{\n    "experiences": [\n        {\n            "full_title": "Daiichi Sankyo Inc, Basking Ridge, NJ - R&D Data Governance Intern    03/2024 - Present",\n            "relevance_score": "high",\n            "points": [\n                "Developed an ICF analysis tool using **BERT**, **T5**, and **Amazon Bedrock**, enhancing language processing capabilities.",\n                "Built a **Flask** frontend for the ICF tool, enabling secure document classification modifications based on user permissions.",\n                "Implemented **Amazon Bedrock** LLMs, increasing classification accuracy by 2

In [257]:
optimized_content.keys()

dict_keys(['skills', 'experience', 'projects'])

In [258]:
optimized_content["projects"]

'```json\n{\n    "projects": [\n        {\n            "name": "Chronic Kidney Disease Predictor",\n            "points": [\n                "Utilized **Python\'s Pandas** and **NumPy** for data preprocessing, ensuring accurate model inputs.",\n                "Implemented **Scikit-learn** to develop a logistic regression model, achieving a 98% F1 score.",\n                "Deployed with **Flask** and **MERN stack**, providing an intuitive platform for disease prediction."\n            ]\n        },\n        {\n            "name": "Flight Price Predictor",\n            "points": [\n                "Enhanced model using **Scikit-learn** and **LSTM networks**, improving forecasting accuracy by 15%.",\n                "Conducted feature engineering with **Pandas**, identifying key price determinants from historical data.",\n                "Deployed on **Heroku** with a **Flask** interface, enabling real-time flight price predictions."\n            ]\n        }\n    ]\n}\n```'

In [259]:
template_style = {
    "name_size": 16,
    "contact_size": 9.5,
    "heading_size": 10.5,
    "content_size": 9
}
doc = generate_resume_docx(cv_data, optimized_content, template_style)


In [260]:
doc

In [261]:
docx_path, pdf_path = save_resume(doc, "Linkedin1", cv_data)

  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
parse_json_content("optimized_content")

ERROR:__main__:Error parsing JSON: Expecting value: line 1 column 1 (char 0)


In [59]:
def parse_json_content(json_str: str) -> Optional[Dict]:
    """
    Safely parse JSON string to dictionary
    """
    try:
        # Remove JSON code block markers if present
        json_str = json_str.replace('```json', '').replace('```', '').strip()
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        logger.error(f"Error parsing JSON: {e}")
        return None


In [68]:
optimized_content={'skills': '```json\n{\n    "Programming Languages": "Java, Python, Object Oriented Programming(Python, Java)",\n    "Frameworks": "PyTorch, Keras, Scikit, Tensorflow, Flask",\n    "Databases": "SQL, MongoDB, MySQL, PostgreSQL, NoSQL",\n    "Technologies": "",\n    "ML Algorithms/Techniques": "Deep Learning, NLP, A/B Testing, Time Series"\n}\n```', 'experience': '[\n    {\n        "title": "Daiichi Sankyo Inc, Basking Ridge, NJ- R&D Data Governance Intern\\t03/2024 - Present",\n        "points": [\n            "Developed ICF analysis tool using BERT and T5 models, enhancing classification accuracy by 20%.",\n            "Built Flask-based frontend for secure document modifications based on user permissions.",\n            "Optimized model accuracy by 20% with Large Language Models using Amazon Bedrock.",\n            "Utilized Amazon SageMaker for scalable legal document processing.",\n            "Improved ICF analysis tool performance with EC2 instances.",\n            "Designed on-premise chatbot reducing access time by 70%.",\n            "Enabled visualizations for improved data comprehension.",\n            "Automated milestone tracking, reducing missed deadlines by 30%.",\n            "Developed secure chatbot architecture, ensuring 100% compliance with regulations.",\n            "Integrated RAG and AI achieving 80% higher answer accuracy.",\n            "Led clinical data migration to Redshift, ensuring data accuracy and analytics accessibility.",\n            "Optimized Redshift query performance for high-speed analysis.",\n            "Implemented data validation checks for migration compliance.",\n            "Aligned tool capabilities with data-sharing policies and compliance standards."\n        ]\n    },\n    {\n        "title": "Rochester Institute Of Technology, Rochester, NY- Research Assistant\\t08/2024 - Present",\n        "points": [\n            "Enhanced federated learning models with gRPC and PyTorch for scalable training algorithms.",\n            "Reduced communication overhead by 15% in distributed environments with PyTorch RPC.",\n            "Accelerated model convergence by 20% with FedDisco algorithm integration.",\n            "Transitioned to adaptive communication strategies for real-time model updates.",\n            "Improved system performance by 30% for Large Language Models.",\n            "Enhanced federated learning model accuracy by 10% using gRPC.",\n            "Designed real-world machine learning projects for student application.",\n            "Provided personalized project support to foster student learning outcomes."\n        ]\n    },\n    {\n        "title": "SEO Content AI, Los Angeles, CA - AI Infrastructure Engineer\\tNov 2022 - July 2023",\n        "points": [\n            "Boosted content generation efficiency by 25% through transformer integration.",\n            "Developed Chrome extension for 40% faster content generation.",\n            "Deployed Docker for scalable cloud deployment.",\n            "Led effective development methodologies for team collaboration.",\n            "Conducted A/B testing on various models for optimized content creation.",\n            "Utilized GCP\'s Google Translate API for multi-language content reach.",\n            "Implemented NLP algorithms for error-free content and user trust."\n        ]\n    },\n    {\n        "title": "White Label Resell, Los Angeles, CA - Machine Learning Engineer\\tJune 2022 - March 2023",\n        "points": [\n            "Achieved 60x cost reduction with automated article generation.",\n            "Analyzed and generated 130K+ high-quality articles within a week.",\n            "Managed large datasets efficiently with DynamoDB.",\n            "Improved content relevance with advanced NLP model tuning.",\n            "Diversified content creation with generative models.",\n            "Optimized specific content generation tasks with T5 Codegen.",\n            "Enhanced thematic accuracy with custom dataset models.",\n            "Streamlined ML development with Git and Docker for continuous improvement."\n        ]\n    },\n    {\n        "title": "Digital Clinics Research and Services, Hyderabad, India - Data Scientist Intern\\tNov 2021 - Dec 2022",\n        "points": [\n            "Developed image classification system with TensorFlow for cancerous cell detection."\n        ]\n    },\n    {\n        "title": "Digital Clinics Research and Services, Hyderabad, India - Data Scientist Intern\\tNov 2021 - Dec 2022",\n        "points": [\n            "Engineered custom segmentation solution with Detectron2 for tumor boundary delineation.",\n            "Implemented YOLOv5 for automated pathological slide screening with OpenCV integration.",\n            "Led MLOps practices with Docker and AWS for scalable AI solutions.",\n            "Mentored junior colleagues on ML and DL projects for enhanced expertise."\n        ]\n    },\n    {\n        "title": "Edgeforce Solutions, Hyderabad, India - Data Scientist Intern\\tNov 2021 - Feb 2022",\n        "points": [\n            "Developed YOLOv5-based real-time object detection system with 90% accuracy."\n        ]\n    }\n]'}

In [217]:
doc = generate_resume_docx(cv_data, optimized_content, template_style)
        

[{'company_name': 'Daiichi Sankyo Inc', 'location': 'Basking Ridge, NJ', 'title': 'R&D Data Governance Intern', 'dates': '03/2024 - Present', 'points': ['Developed ICF analysis tool with BERT, T5, increasing classification accuracy by 20%', 'Built Flask frontend for secure document classification based on user permissions', 'Utilized Amazon SageMaker for scalable model training on legal documents', 'Optimized model inference with EC2, enhancing tool performance and reliability']}, {'company_name': 'Rochester Institute Of Technology', 'location': 'Rochester, NY', 'title': 'Research Assistant', 'dates': '08/2024 - Present', 'points': ['Enhanced federated learning using gRPC and PyTorch, reducing communication overhead by 15%', 'Integrated FedDisco algorithm, accelerating model convergence and lowering training time by 20%', 'Developed distributed LLM training techniques, boosting system performance by 30%', 'Leveraged gRPC for secure communication, improving model accuracy by 10%']}, {'c

TypeError: string indices must be integers

In [231]:
def create_resume_prompts(cv_data: Dict, job_description: str) -> Dict[str, str]:
    """
    Create prompts for resume sections including company name extraction
    """
    prompts = {
        "company_extraction": """
        Extract the exact company name from this job description.
        If no company name is found, return 'Company'.
        Return only the company name without any additional text.
        
        Job Description: {job_description}
        """,
        
        "skills": f"""
        Given these skills and job description, return the most relevant ones maintaining the same categories.
        Each line should be 10-16 words maximum.
        
        Current Skills: {cv_data['Skills']}
        Job Description: {job_description}
        
        Format as JSON:
        {{
            "Programming Languages": "...",
            "Frameworks": "...",
            "Databases": "...",
            "Technologies": "...",
            "ML Algorithms/Techniques": "..."
        }}
        """,
        
        "experience": f"""
        Analyze the job description first to identify the most relevant experience. Then optimize the experiences using STAR method.
        
        Rules:
        1. Format each experience exactly as: "Company Name, Location - Title    Date"
        2. Points distribution:
           - Most relevant experience: 4 points
           - Moderately relevant: 3 points
           - Less relevant: 2 points
        3. Each bullet point must be 12-20 words and include:
           - Action verb
           - Technical tools/skills/frameworks used (bold these)
           - Quantifiable impact/metrics
        
        Current Experience: {cv_data['Professional Experience And Internships']}
        Job Description: {job_description}
        
        Return as JSON:
        {{
            "experiences": [
                {{
                    "full_title": "Company Name, Location - Title    Date",
                    "relevance_score": "high/medium/low",
                    "points": ["point1", "point2", ...]
                }}
            ]
        }}
        """,
        
        "projects": f"""
        Select and optimize 2 most relevant projects based on the job description requirements.
        
        Rules:
        1. Select projects that best demonstrate skills mentioned in job description
        2. 2-3 bullet points per project (10-20 words each)
        3. Each point must include:
           - Technical tools/frameworks used (bold these)
           - Implementation details
           - Measurable outcomes
        
        Current Projects: {cv_data['Projects']}
        Job Description: {job_description}
        
        Return as JSON:
        {{
            "projects": [
                {{
                    "name": "Project Name",
                    "points": [
                        "Used **Tool/Framework** to implement... resulting in X% improvement"
                    ]
                }}
            ]
        }}
        """
    }
    return prompts

In [252]:


def generate_resume_docx(cv_data: Dict, optimized_content: Dict, template_style: Dict) -> Document:
    """
    Generate resume DOCX with improved formatting
    """
    doc = Document()
    
    # Set margins
    for section in doc.sections:
        section.top_margin = Inches(0.3)  # Reduced margins
        section.bottom_margin = Inches(0.3)
        section.left_margin = Inches(0.5)
        section.right_margin = Inches(0.5)

    # Name
    name_paragraph = doc.add_paragraph()
    name_run = name_paragraph.add_run(cv_data["Name"])
    name_run.font.size = Pt(16)
    name_run.font.bold = True
    name_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Contact Info with hyperlinks
    contact_paragraph = doc.add_paragraph()
    contact_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Add email and phone
    contact_run = contact_paragraph.add_run(f"{cv_data['Contact']['email']} | {cv_data['Contact']['phone']} | ")
    contact_run.font.size = Pt(10.5)
    
    # Add hyperlinks
    for platform, url in cv_data["Contact"]["links"].items():
#         if url:
#             hyperlink = contact_paragraph.add_hyperlink(url, platform)
#             hyperlink.font.size = Pt(10.5)
#             if platform != list(cv_data["Contact"]["links"].keys())[-1]:
#                 contact_paragraph.add_run(" | ")
        if url:
            add_hyperlink(contact_paragraph, url, platform)  # Use the helper function
            if platform != list(cv_data["Contact"]["links"].keys())[-1]:
                contact_paragraph.add_run(" | ")

#     def add_section_heading(title):
#         """Add section heading with blue color and separator line"""
#         paragraph = doc.add_paragraph()
#         paragraph.space_before = Pt(6)  # Reduced spacing
#         paragraph.space_after = Pt(3)
#         run = paragraph.add_run(title)
#         run.font.size = Pt(11.5)
#         run.font.bold = True
#         run.font.color.rgb = RGBColor(0, 0, 139)  # Dark blue
        
#         # Add separator line
#         border = paragraph.paragraph_format.border
#         border.bottom_style = WD_BORDER.SINGLE
#         border.bottom_color.rgb = RGBColor(0, 0, 139)
    def add_section_heading(title):
        """Add section heading with blue color and separator line"""
        paragraph = doc.add_paragraph()
        paragraph.space_before = Pt(0)  # Further reduced spacing
        paragraph.space_after = Pt(0)
        run = paragraph.add_run(title)
        run.font.size = Pt(11.5)
        run.font.bold = True
        run.font.color.rgb = RGBColor(0, 0, 139)

        # Add separator line
        separator = doc.add_paragraph()
        separator_format = separator.paragraph_format
        separator_format.space_before = Pt(0)
        separator_format.space_after = Pt(0)
        separator_run = separator.add_run("_" * 150)
        separator_run.font.color.rgb = RGBColor(0, 0, 139)
        separator_run.font.size = Pt(8)

    # Education section with aligned dates
    add_section_heading("EDUCATION")
    for edu in cv_data["Education"]:
        p = doc.add_paragraph()
        if "Coursework:" in edu:
            # Split into main education info and coursework
            main_part, coursework_part = edu.split("Coursework:", 1)

            # Handle main part (institution, degree, date)
            if "GPA:" in main_part:
                info, gpa = main_part.rsplit("GPA:", 1)
                p.add_run(info).bold = True
                p.add_run("GPA:").bold = False
                p.add_run(gpa.strip()).bold = False
            else:
                p.add_run(main_part).bold = True

            # Add coursework (unbolded)
            p.add_run("Coursework:").bold = False
            p.add_run(coursework_part.strip()).bold = False
        else:
            # If no coursework, just handle the main part
            if "GPA:" in edu:
                info, gpa = edu.rsplit("GPA:", 1)
                p.add_run(info).bold = True
                p.add_run("GPA:").bold = False
                p.add_run(gpa.strip()).bold = False
            else:
                p.add_run(edu).bold = True

        
    p.paragraph_format.space_after = Pt(0)
    # Skills
    add_section_heading("SKILLS")
    skills_data = parse_json_content(optimized_content["skills"])
    if skills_data:
        for category, skills in skills_data.items():
            if skills:
                p = doc.add_paragraph()
                p.add_run(f"{category}: ").bold = True
                p.add_run(skills)
                p.paragraph_format.space_after = Pt(1)

# Experience section with proper formatting
    add_section_heading("PROFESSIONAL EXPERIENCE AND INTERNSHIPS")
    experience_data = parse_json_content(optimized_content["experience"])
    if experience_data and 'experiences' in experience_data:
        for exp in experience_data['experiences']:
            p = doc.add_paragraph()
            p.add_run(exp['full_title']).bold = True
            p.paragraph_format.space_after = Pt(1)
            
            # Add points based on relevance score
            max_points = 4 if exp['relevance_score'] == 'high' else (2 if exp['relevance_score'] == 'medium' else 2)
            for point in exp['points'][:max_points]:
                bullet_p = doc.add_paragraph(style='List Bullet')
                bullet_p.paragraph_format.left_indent = Inches(0.25)
                bullet_p.paragraph_format.space_after = Pt(1)
#                 bullet_p.add_run(point)
                # Handle bold text in points
                parts = point.split('**')
                for i, part in enumerate(parts):
                    run = bullet_p.add_run(part)
                    if i % 2 == 1:  # Bold text between **
                        run.bold = True

    # Projects section
    add_section_heading("PROJECTS")
    projects_data = parse_json_content(optimized_content["projects"])
    if projects_data and 'projects' in projects_data:
        for project in projects_data['projects']:
            p = doc.add_paragraph()
            p.add_run(project['name']).bold = True
            p.paragraph_format.space_after = Pt(1)
            
            for point in project['points']:
                bullet_p = doc.add_paragraph(style='List Bullet')
                bullet_p.paragraph_format.left_indent = Inches(0.25)
                bullet_p.paragraph_format.space_after = Pt(1)
                # Handle bold text in points
                parts = point.split('**')
                for i, part in enumerate(parts):
                    run = bullet_p.add_run(part)
                    if i % 2 == 1:  # Bold text between **
                        run.bold = True

    return doc
def process_complete_resume(cv_data: Dict, job_description: str, client: OpenAI) -> Tuple[Optional[str], Optional[str]]:
    """
    Complete resume generation process with error handling
    """
    try:
        # Get prompts
        prompts = create_resume_prompts(cv_data, job_description)
        
        # Extract company name
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {"role": "system", "content": "You are a job description analyzer."},
                {"role": "user", "content": prompts["company_extraction"].format(job_description=job_description)}
            ]
        )
        company_name = response.choices[0].message.content.strip()
        
        # Get optimized content
        optimized_content = {}
        for section in ["skills", "experience","projects"]:
            response = client.chat.completions.create(
#                 model="gpt-3.5-turbo-16k",
                  model="o1-mini-2024-09-12",

                messages=[
                    {"role": "assistant", "content": "You are an expert resume writer."},
                    {"role": "user", "content": prompts[section]}
                ],
                max_completion_tokens=10000,
#                 reasoning_effort="high"

                
            )
            optimized_content[section] = response.choices[0].message.content
        
        # Template styling
        template_style = {
            "name_size": 16,
            "contact_size": 10.5,
            "heading_size": 11.5,
            "content_size": 10
        }
        print("Optimized Content")
        print(optimized_content)
        print("-----------")      
        return optimized_content # Generate document
#         doc = generate_resume_docx(cv_data, optimized_content, template_style)
#         if not doc:
#             logger.error("Failed to generate resume document")
#             return None, None
        
#         # Save both versions
#         try:
#             docx_path, pdf_path = save_resume(doc, company_name, cv_data)
#             return docx_path, pdf_path
#         except Exception as e:
#             logger.error(f"Error saving resume: {e}")
#             return None, None
            
    except Exception as e:
        logger.error(f"Error in resume generation process: {e}")
        return None, None

In [116]:
cv_data = extract_cv_data(doc)  # Using the extract_cv_data function from earlier
cv_data

linkedin  linkedin https://www.linkedin.com/in/mohammad-faseeh-ahmed/
https://www.linkedin.com/in/mohammad-faseeh-ahmed/
github github https://github.com/faseehahmed26
https://github.com/faseehahmed26
portfolio portfolio https://faseehahmed26.github.io/portfolio/
https://faseehahmed26.github.io/portfolio/
kaggle kaggle https://www.kaggle.com/faseeh001
https://www.kaggle.com/faseeh001
tableau tableau https://public.tableau.com/app/profile/faseeh5112
https://public.tableau.com/app/profile/faseeh5112


{'Name': 'Mohammad Faseeh Ahmed',
 'Contact': {'email': 'mm9314@g.rit.edu',
  'phone': '+1 585 202 5217',
  'links': {'LinkedIn': 'https://www.linkedin.com/in/mohammad-faseeh-ahmed/',
   'Github': 'https://github.com/faseehahmed26',
   'Portfolio': 'https://faseehahmed26.github.io/portfolio/',
   'Kaggle': 'https://www.kaggle.com/faseeh001',
   'Tableau': 'https://public.tableau.com/app/profile/faseeh5112'}},
 'Education': ['Rochester Institute of Technology, Rochester, NY,  M.S in Data Science\t      Expected May 2025',
  'Coursework: Neural Networks, Software Engineering for Data Science, Applied Statistics.                         GPA: 3.84/4.00',
  'Jawaharlal Nehru Technological University Hyderabad, B.Tech in Computer Science\t July 2018 - July 2022',
  'Coursework: Data Structures and Algorithms, Computer Vision, Artificial Intelligence, NLP \t         GPA: 3.2/4.00'],
 'Skills': {'Programming Languages': 'Java, Python, C++, R, JavaScript, Object Oriented Programming(Python, Jav

In [115]:
def extract_cv_data(doc: Document) -> Dict:
    """
    Extract and structure CV data from document
    """
    cv_data = {
        "Name": "",
        "Contact": {
            'email': '',
            'phone': '',
            'links':{'LinkedIn': '',
            'Github': '',
            'Portfolio': '',
            'Kaggle': '',
            'Tableau': ''}
        },
        "Education": [],
        "Skills": {
            "Programming Languages": "",
            "Frameworks": "",
            "Databases": "",
            "Technologies": "",
            "ML Algorithms/Techniques": ""
        },
        "Professional Experience And Internships": [],
        "Projects": [],
        "Research Papers": []
    }
    
    # Initialize current section
    current_section = None
    
    for idx, para in enumerate(doc.paragraphs):
        text = para.text.strip()
        if not text:
            continue

        # Track sections
        if "EDUCATION" in text:
            current_section = "education"
            continue
        elif "SKILLS" in text:
            current_section = "skills"
            continue
        elif "PROFESSIONAL EXPERIENCE" in text:
            current_section = "experience"
            continue
        elif "PROJECTS" in text:
            current_section = "projects"
            continue
        elif "RESEARCH PAPERS" in text:
            current_section = "research"
            continue

        # Extract Name
        if idx == 0:
            cv_data["Name"] = text
            continue

        # Extract Contact Info and Links
        if idx == 1:
            parts = text.split(' | ')
            cv_data["Contact"]["email"] = parts[0]
            cv_data["Contact"]["phone"] = parts[1]
#                         elif any(x in hyperlink.text.lower() for x in ['linkedin', 'github', 'portfolio', 'kaggle', 'tableau']):
#                 static_content['contact_info']['links'][hyperlink.text.lower()] = hyperlink.address
            # Extract hyperlinks
            if para.hyperlinks:
                for hyperlink in para.hyperlinks:
                    for platform in ['LinkedIn', 'Github', 'Portfolio', 'Kaggle', 'Tableau']:
#                         print(hyperlink.text.lower(),platform.lower(), hyperlink.address)
#                         print(str(platform.lower()) in str(hyperlink.text.lower()).strip())
                        if str(platform.lower()) in  str(hyperlink.text.lower()).strip():
                            print(hyperlink.text.lower(),platform.lower(), hyperlink.address)
                            cv_data["Contact"]['links'][platform] = hyperlink.address
                            print(cv_data["Contact"]['links'][platform])
            continue

        # Process sections
        if current_section == "education":
            if any(x in text for x in ["Institute", "University", "GPA"]):
                cv_data["Education"].append(text)
                
        elif current_section == "skills":
            if text.startswith("Programming Languages:"):
                cv_data["Skills"]["Programming Languages"] = text.split(": ")[1]
            elif text.startswith("Frameworks:"):
                cv_data["Skills"]["Frameworks"] = text.split(": ")[1]
            elif text.startswith("Databases:"):
                cv_data["Skills"]["Databases"] = text.split(": ")[1]
            elif text.startswith("Technologies:"):
                cv_data["Skills"]["Technologies"] = text.split(": ")[1]
            elif text.startswith("ML Algorithms"):
                cv_data["Skills"]["ML Algorithms/Techniques"] = text.split(": ")[1]
                
        elif current_section == "experience":
            if any(x in text for x in ["Intern", "Engineer"]) and not text.startswith("•"):
                cv_data["Professional Experience And Internships"].append({
                    "title": text,
                    "points": []
                })
            elif cv_data["Professional Experience And Internships"] and not text.isupper():
                cv_data["Professional Experience And Internships"][-1]["points"].append(text)
                
        elif current_section == "projects":
            if not text.startswith("•") and not text.isupper():
                cv_data["Projects"].append({
                    "name": text,
                    "points": []
                })
            elif cv_data["Projects"] and not text.isupper():
                cv_data["Projects"][-1]["points"].append(text)
                
        elif current_section == "research":
            if text.endswith(":"):
                cv_data["Research Papers"].append({
                    "title": text,
                    "description": ""
                })
            elif cv_data["Research Papers"]:
                cv_data["Research Papers"][-1]["description"] = text

    return cv_data

In [163]:
def create_resume_prompts(cv_data: Dict, job_description: str) -> Dict[str, str]:
    """
    Create prompts for resume sections including company name extraction
    """
    prompts = {
        "company_extraction": """
        Extract the exact company name from this job description.
        If no company name is found, return 'Company'.
        Return only the company name without any additional text.
        
        Job Description: {job_description}
        """,
        
        "skills": f"""
        Given these skills and job description, return the most relevant ones maintaining the same categories.
        Each line should be 10-16 words maximum.
        
        Current Skills: {cv_data['Skills']}
        Job Description: {job_description}
        
        Format as JSON:
        {{
            "Programming Languages": "...",
            "Frameworks": "...",
            "Databases": "...",
            "Technologies": "...",
            "ML Algorithms/Techniques": "..."
        }}
        """,
        
        "experience": f"""
        Optimize these experiences for the job description using STAR method.
        - Keep company names, titles, and dates exact
        - 2-4 bullet points per role (10-16 words each)
        - Include technical details and metrics
        
        Current Experience: {cv_data['Professional Experience And Internships']}
        Job Description: {job_description}
        
        Format as JSON with exact dates and titles.
        """
    }
    return prompts

def generate_resume_docx(cv_data: Dict, optimized_content: Dict, template_style: Dict) -> Document:
    """
    Generate resume DOCX with specific formatting
    """
    doc = Document()
    
    # Set margins
    for section in doc.sections:
        section.top_margin = Inches(0.5)
        section.bottom_margin = Inches(0.5)
        section.left_margin = Inches(0.5)
        section.right_margin = Inches(0.5)

    # Name
    name_paragraph = doc.add_paragraph()
    name_run = name_paragraph.add_run(cv_data["Name"])
    name_run.font.size = Pt(16)
    name_run.font.bold = True
    name_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Contact Info with hyperlinks
    contact_paragraph = doc.add_paragraph()
    contact_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Add email and phone
    contact_run = contact_paragraph.add_run(f"{cv_data['Contact']['email']} | {cv_data['Contact']['phone']} | ")
    contact_run.font.size = Pt(10.5)
    
    # Add hyperlinked items
    for platform, url in cv_data["Contact"]["links"].items():
        if url:
            hyperlink = contact_paragraph.add_run(platform)
            hyperlink.font.size = Pt(10.5)
            # Add separator except for last item
            if platform != list(cv_data["Contact"]["links"].keys())[-1]:
                contact_paragraph.add_run(" | ")

    # Add sections
    def add_section_heading(title):
        doc.add_paragraph().add_run().add_break()
        heading = doc.add_paragraph()
        run = heading.add_run(title)
        run.font.size = Pt(11.5)
        run.font.bold = True
        return heading

    # Education
    add_section_heading("EDUCATION")
    for edu in cv_data["Education"]:
        p = doc.add_paragraph()
        p.add_run(edu).font.size = Pt(10)

    # Skills
    add_section_heading("SKILLS")
    for category, skills in optimized_content["skills"].items():
        p = doc.add_paragraph()
        run = p.add_run(f"{category}: {skills}")
        run.font.size = Pt(10)

    # Experience
    add_section_heading("PROFESSIONAL EXPERIENCE AND INTERNSHIPS")
    for exp in optimized_content["experience"]:
        # Add company and title
        p = doc.add_paragraph()
        run = p.add_run(exp["title"])
        run.font.size = Pt(10)
        
        # Add bullet points
        for point in exp["points"]:
            p = doc.add_paragraph(point, style='List Bullet')
            p.paragraph_format.left_indent = Inches(0.25)
            p.runs[0].font.size = Pt(10)

    return doc

def save_resume(doc: Document, company_name: str, cv_data: Dict, output_dir: str = "resumes"):
    """
    Save resume in both DOCX and PDF formats
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate base filename
    base_name = cv_data["Name"].lower().replace(' ', '_')
    company_name = company_name.lower().replace(' ', '_')
    base_filename = f"{base_name}_{company_name}_resume"
    
    # Save DOCX
    docx_path = os.path.join(output_dir, f"{base_filename}.docx")
    doc.save(docx_path)
    
    # Convert to PDF
    pdf_path = os.path.join(output_dir, f"{base_filename}.pdf")
    convert(docx_path, pdf_path)
    
    return docx_path, pdf_path

def process_complete_resume(cv_data: Dict, job_description: str, client: OpenAI):
    """
    Complete resume generation process
    """
    # Get prompts
    prompts = create_resume_prompts(cv_data, job_description)
    
    # Extract company name
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a job description analyzer."},
            {"role": "user", "content": prompts["company_extraction"].format(job_description=job_description)}
        ]
    )
    company_name = response.choices[0].message.content.strip()
    
    # Get optimized content
    optimized_content = {}
    for section in ["skills", "experience"]:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16K",
            messages=[
                {"role": "system", "content": "You are an expert resume writer."},
                {"role": "user", "content": prompts[section]}
            ],
            max_tokens=10000
            
        )
        optimized_content[section] = response.choices[0].message.content
    
    # Template styling
    template_style = {
        "name_size": 16,
        "contact_size": 10.5,
        "heading_size": 11.5,
        "content_size": 10
    }
    print("Optimized Content")
    print(optimized_content)
    print("-----------")
    # Generate document
    doc = generate_resume_docx(cv_data, optimized_content, template_style)
    
    # Save both versions
    docx_path, pdf_path = save_resume(doc, company_name, cv_data)
    
    return docx_path, pdf_path

# Example usage
   